## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-16-56-00 +0000,wsj,"Dollar Could Stay Steady For Now, More Likely ...",https://www.wsj.com/opinion/asian-currencies-c...
1,2026-01-09-16-53-58 +0000,nyt,Renee Good’s Time at Old Dominion Included an ...,https://www.nytimes.com/2026/01/08/us/renee-go...
2,2026-01-09-16-53-41 +0000,nyt,"U.S. Hiring Continues at Modest Pace, but Weak...",https://www.nytimes.com/2026/01/05/business/ec...
3,2026-01-09-16-52-34 +0000,nyt,Live Updates: U.S. Forces Board Another Tanker...,https://www.nytimes.com/live/2026/01/09/world/...
4,2026-01-09-16-51-12 +0000,wapo,Khamenei says Iran won’t ‘back down’ amid mass...,https://www.washingtonpost.com/world/2026/01/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
83,trump,80
70,venezuela,31
40,ice,27
41,shooting,20
28,oil,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
230,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,176
272,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,162
100,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...,161
382,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,156
160,2026-01-09-09-00-00 +0000,cbc,How Trump and Vance's accounts of Minneapolis ...,https://www.cbc.ca/news/world/minneapolis-ice-...,153


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
230,176,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
19,84,2026-01-09-16-28-18 +0000,startribune,Live: Minnesota officials rail against federal...,https://www.startribune.com/ice-raids-minnesot...
272,82,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
181,57,2026-01-09-05-04-26 +0000,nypost,Russia fires nuclear-capable Oreshnik missile ...,https://nypost.com/2026/01/09/world-news/russi...
194,55,2026-01-09-03-08-27 +0000,nypost,Trump plans to meet Venezuelan opposition lead...,https://nypost.com/2026/01/08/us-news/trump-pl...
85,53,2026-01-09-13-37-11 +0000,nyt,Iran Is Cut Off From Internet as Protests Call...,https://www.nytimes.com/2026/01/08/world/middl...
95,51,2026-01-09-12-49-16 +0000,nypost,ICE agent who shot Renee Nicole Good in Minnea...,https://nypost.com/2026/01/09/us-news/jonathan...
16,41,2026-01-09-16-31-00 +0000,wsj,American employers added a seasonally adjusted...,https://www.wsj.com/economy/jobs/jobs-report-d...
49,41,2026-01-09-15-13-22 +0000,nypost,US nabs another ‘ghost fleet’ tanker in Caribb...,https://nypost.com/2026/01/09/us-news/us-nabs-...
131,41,2026-01-09-11-00-00 +0000,latimes,California is completely drought-free for the ...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
